# Project database

In [7]:
# import libraries
import polars as pl
import pandas as pd
import geopandas as gpd
import duckdb as db
import warnings
warnings.filterwarnings('ignore')

In [8]:
gpd.read_parquet('geo.parquet')

,CVEGEO,CVE_ENT,CVE_MUN,NOMGEO,NOM_ENT,COV_,COV_ID,AREA,PERIMETER,geometry
0,01008,01,008,San José de Gracia,Aguascalientes,0,1,857.790574,181.537599,"POLYGON ((-102.45537 22.31212, -102.45501 22.3..."
1,01009,01,009,Tepezalá,Aguascalientes,1,2,229.661012,89.329532,"POLYGON ((-102.17738 22.36243, -102.17968 22.3..."
2,01007,01,007,Rincón de Romos,Aguascalientes,2,3,372.295163,161.749880,"POLYGON ((-102.22684 22.37393, -102.22661 22.3..."
3,01006,01,006,Pabellón de Arteaga,Aguascalientes,3,4,195.735946,105.612997,"POLYGON ((-102.25345 22.18302, -102.2513 22.18..."
4,01005,01,005,Jesús María,Aguascalientes,4,5,499.895092,163.500811,"POLYGON ((-102.35391 22.06255, -102.35312 22.0..."
...,...,...,...,...,...,...,...,...,...,...
2470,32029,32,029,Miguel Auza,Zacatecas,2470,2471,1093.921188,183.913765,"POLYGON ((-103.43211 24.35769, -103.43133 24.3..."
2471,32028,32,028,Mezquital del Oro,Zacatecas,2471,2472,482.008504,146.234897,"POLYGON ((-103.28714 21.34599, -103.28698 21.3..."
2472,32011,32,011,Trinidad García de la Cadena,Zacatecas,2472,2473,304.265670,105.480166,"POLYGON ((-103.4353 21.3292, -103.43494 21.329..."
2473,32027,32,027,Melchor Ocampo,Zacatecas,2473,2474,1862.048602,247.933775,"POLYGON ((-102.29548 25.12519, -102.26655 25.1..."


In [2]:
# create or connect to database
conn = db.connect('homicides.dudkdb')

## Upload geojson table

In [3]:
# install and load duckdb spatial extension
conn.execute('INSTALL spatial;')
conn.execute('LOAD spatial;')

In [4]:
# create geo table
conn.execute('''
    create or replace table geodata AS
        select * 
        from 'geo.parquet'
''')

In [5]:
# geo query
query = '''
    SELECT * exclude geometry,
        ST_AsWKB(geometry) AS geometry
    FROM geodata
'''

In [6]:
# read query with geopandas
gdf = gpd.read_postgis(query, conn, geom_col='geometry')

## Upload homicides and population tables

In [62]:
conn.sql('''
    create or replace table homicides as
        select *
        from homicides.csv
''')

In [66]:
conn.sql('''
    create or replace table population as
        select * from pop.csv
''')

In [12]:
conn.sql('show tables')

┌────────────┐
│    name    │
│  varchar   │
├────────────┤
│ geodata    │
│ homicides  │
│ population │
└────────────┘

In [13]:
# close connection
conn.close()